In [11]:
import _thread, time, socket, json
import numpy as np
from IPython.display import clear_output

In [7]:
def imu_ctrl_main(threadName, delay):
    count = 0
    print("Thread starts: ", threadName)
    while count < 5:
        time.sleep(delay)
        count += 1
        print('imu')



In [8]:
def cv_main(threadName, delay):
    count = 0
    print("Thread starts: ", threadName)
    while count < 5:
        time.sleep(delay)
        count += 1
        print('cv')

In [10]:
try:
    _thread.start_new_thread(imu_ctrl_main, ("imu_ctrl_thread", 1, ))
    _thread.start_new_thread(cv_main, ("cv_thread", 2, ))
except:
    print("Error: to start thread")



Thread starts:  imu_ctrl_thread
Thread starts:  cv_thread
imu
cvimu

imu
imu
cv
imu
cv
cv
cv
